In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import datasets, transforms

from tqdm import tqdm
import random, time, os, sys

In [3]:
import sparse_nonlinear_lib as snl

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cpu")

## For FMNIST dataset

In [5]:
train_transform = transforms.Compose([
            transforms.ToTensor(),
        ])
test_transform = transforms.Compose([
            transforms.ToTensor(),
        ])

train_dataset = datasets.FashionMNIST(root="../../../../_Datasets/FMNIST/", train=True, download=True, transform=train_transform)
test_dataset = datasets.FashionMNIST(root="../../../../_Datasets/FMNIST/", train=False, download=True, transform=test_transform)

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=50, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=50, shuffle=False, num_workers=2)

In [7]:
## demo of train loader
xx, yy = iter(train_loader).next()
xx.shape

torch.Size([50, 1, 28, 28])

# Model Comparision

## Pair Linear Mixing

In [8]:
def img2patch(x, input_dim=(1, 28, 28), patch_size=(7, 4)):
    y = nn.functional.unfold(x, 
                             kernel_size=patch_size, 
                             stride=patch_size
                            )
    return y

In [9]:
def patch2img(x, patch_size=(7, 4), input_dim=(1, 28, 28)):
    y = nn.functional.fold(x, (input_dim[-2], input_dim[-1]), 
                               kernel_size=patch_size, 
                               stride=patch_size
                              )
    return y

1. Linearize by expanding the dimension of folded image.

## Final Model

In [241]:
class FMNIST_BlockMLP(nn.Module):
    
    def __init__(self, img_size=(1, 28, 28), select=1024, block_size=2, hidden_layers_ratio=[4], actf=nn.ELU):
        super().__init__()
        self.dim_sel = snl.DimensionSelector(np.prod(img_size), select)

        self.block_mlp = snl.BlockMLP_MixerBlock(select, block_size, 
                                                 hidden_layers_ratio=hidden_layers_ratio, actf=actf)
        self.bn = nn.BatchNorm1d(select)
        self.actf = nn.ELU()
        self.fc = nn.Linear(select, 10)
        
    def forward(self, x):
        bs = x.shape[0]
        x = x.reshape(bs, -1)
        x = self.dim_sel(x)
        x = self.block_mlp(x)
        x = self.bn(x)
        x = self.actf(x)
        x = self.fc(x)
        return x

In [255]:
model = FMNIST_BlockMLP(block_size=32, hidden_layers_ratio=[16])
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  2144266


In [249]:
##### USING Hidden_Dim to change PARAMETERS
# class FMNIST_OrdMLP(nn.Module):
    
#     def __init__(self, img_size=(1, 28, 28), hidden_size=1024):
#         super().__init__()
#         self.input_dim = np.prod(img_size)
#         M = hidden_size #354 #1024
#         self.l0 = nn.Linear(self.input_dim, M)
#         self.bn = nn.BatchNorm1d(M)
#         self.elu = nn.ELU()
#         self.l1 = nn.Linear(M, 10)
        
#     def forward(self, x):
#         bs = x.shape[0]
#         x = x.reshape(bs, -1)
#         x = self.l0(x)
#         x = self.bn(x)
#         x = self.elu(x)
#         x = self.l1(x)
#         return x

# model = FMNIST_OrdMLP(hidden_size=688)

#### USING DimensionSelector to make comparative

class FMNIST_OrdMLP(nn.Module):
    
    def __init__(self, img_size=(1, 28, 28), select=1024):
        super().__init__()
        self.input_dim = np.prod(img_size)
        self.dim_sel = snl.DimensionSelector(np.prod(img_size), select)
        
        self.l0 = nn.Linear(select, select)
        self.bn = nn.BatchNorm1d(select)
        self.actf = nn.ELU()
        self.l1 = nn.Linear(select, 10)
        
    def forward(self, x):
        bs = x.shape[0]
        x = x.reshape(bs, -1)
        x = self.l0(x)
        x = self.bn(x)
        x = self.actf(x)
        x = self.l1(x)
        return x

In [250]:
model = FMNIST_OrdMLP(select=1024)
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  1061898


In [214]:
class FMNIST_SparseMLP(nn.Module):
    
    def __init__(self, img_size=(1, 28, 28), select=1024, block_size=2):
        super().__init__()
        self.dim_sel = snl.DimensionSelector(np.prod(img_size), select)
        
        self.l0 = snl.BlockLinear_MixerBlock(select, block_size)
        self.bn = nn.BatchNorm1d(select)
        self.actf = nn.ELU()
        self.l1 = nn.Linear(select, 10)
        
    def forward(self, x):
        bs = x.shape[0]
        x = x.reshape(bs, -1)
        x = self.dim_sel(x)
        x = self.l0(x)
        x = self.bn(x)
        x = self.actf(x)
        x = self.l1(x)
        return x

In [215]:
model = FMNIST_SparseMLP(block_size=2).to(device)
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  32778


In [216]:
model(torch.randn(2, 1, 28, 28).to(device)).shape

torch.Size([2, 10])

In [217]:
class FMNIST_PairBilinear(nn.Module):
    
    def __init__(self, img_size=(1, 28, 28), select=1024, grid_width=5):
        super().__init__()

        self.dim_sel = snl.DimensionSelector(np.prod(img_size), select)
        self.block_func = snl.PairBilinear_MixerBlock(select, select, grid_width=grid_width)
        self.bn = nn.BatchNorm1d(select)
        self.actf = nn.ELU()
        self.fc = nn.Linear(select, 10)
        
    def forward(self, x):
        bs = x.shape[0]
        x = x.reshape(bs,-1)
        x = self.dim_sel(x)
        x = self.block_func(x)
        x = self.bn(x)
        x = self.actf(x)
        x = self.fc(x)
        return x

In [218]:
model = FMNIST_PairBilinear(grid_width=10)
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  1057802


## Create Models

In [219]:
# model = FMNIST_BlockMLP(img_size=(1, 28, 28), patch_size=(7, 7))
# model = FMNIST_OrdMLP(img_size=(1, 28, 28))
# model = FMNIST_PairBilinear(img_size=(1, 28, 28), grid_width=7)



In [227]:
model = model.to(device)
model

FMNIST_BlockMLP(
  (dim_sel): DimensionSelector: [+=240]
  (block_mlp): BlockMLP_MixerBlock(
    (facto_nets): ModuleList(
      (0): BlockMLP(
        (mlp): Sequential(
          (0): BlockLinear: [torch.Size([512, 2, 8])]
          (1): ELU(alpha=1.0)
          (2): BlockLinear: [torch.Size([512, 8, 2])]
        )
      )
      (1): BlockMLP(
        (mlp): Sequential(
          (0): BlockLinear: [torch.Size([512, 2, 8])]
          (1): ELU(alpha=1.0)
          (2): BlockLinear: [torch.Size([512, 8, 2])]
        )
      )
      (2): BlockMLP(
        (mlp): Sequential(
          (0): BlockLinear: [torch.Size([512, 2, 8])]
          (1): ELU(alpha=1.0)
          (2): BlockLinear: [torch.Size([512, 8, 2])]
        )
      )
      (3): BlockMLP(
        (mlp): Sequential(
          (0): BlockLinear: [torch.Size([512, 2, 8])]
          (1): ELU(alpha=1.0)
          (2): BlockLinear: [torch.Size([512, 8, 2])]
        )
      )
      (4): BlockMLP(
        (mlp): Sequential(
          (0)

In [228]:
model(torch.randn(2, 1, 28, 28).to(device)).shape

torch.Size([2, 10])

In [229]:
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  227338


## Training

In [230]:
 ## debugging to find the good classifier/output distribution.
# model_name = 'block_mlp_mixer_fmnist_v0'
# model_name = 'ord_mlp_mixer_fmnist_v0'
model_name = 'pair_bilinear_mixer_fmnist_v0'

In [236]:
EPOCHS = 50
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.00003)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [237]:
## Following is copied from 
### https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

# Training
def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print(f"[Train] {epoch} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    return

In [238]:
best_acc = -1
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/{model_name}.pth')
        best_acc = acc

In [239]:
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
resume = False

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('./models'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load(f'./models/{model_name}.pth')
    model.load_state_dict(checkpoint['model'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

In [240]:
### Train the whole damn thing

for epoch in range(start_epoch, start_epoch+EPOCHS): ## for 200 epochs
    train(epoch)
    test(epoch)
    scheduler.step()

100%|████████████████████████████████████████████████| 1200/1200 [00:07<00:00, 155.39it/s]


[Train] 0 Loss: 0.447 | Acc: 83.993 50396/60000


100%|██████████████████████████████████████████████████| 200/200 [00:00<00:00, 386.59it/s]


[Test] 0 Loss: 0.420 | Acc: 84.980 8498/10000
Saving..


100%|████████████████████████████████████████████████| 1200/1200 [00:07<00:00, 157.69it/s]


...

[Train] 44 Loss: 0.056 | Acc: 98.275 58965/60000


100%|██████████████████████████████████████████████████| 200/200 [00:00<00:00, 377.29it/s]


[Test] 44 Loss: 0.409 | Acc: 89.280 8928/10000


 91%|███████████████████████████████████████████▊    | 1094/1200 [00:06<00:00, 158.78it/s]


KeyboardInterrupt: 

In [ ]:
best_acc

In [ ]:
checkpoint = torch.load(f'./models/{model_name}.pth')
best_acc = checkpoint['acc']
start_epoch = checkpoint['epoch']

best_acc, start_epoch

In [49]:
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [50]:
model

FMNIST_PairBilinear(
  (patch_linear): Linear(in_features=49, out_features=64, bias=True)
  (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block_func): PairBilinear_MixerBlock(
    (selector): BiasLayer: [1024]
    (pairwise_mixing): ModuleList(
      (0): PairLinear: [1024 -> 1024] (grid: 7)
      (1): PairLinear: [1024 -> 1024] (grid: 7)
      (2): PairLinear: [1024 -> 1024] (grid: 7)
      (3): PairLinear: [1024 -> 1024] (grid: 7)
      (4): PairLinear: [1024 -> 1024] (grid: 7)
      (5): PairLinear: [1024 -> 1024] (grid: 7)
      (6): PairLinear: [1024 -> 1024] (grid: 7)
      (7): PairLinear: [1024 -> 1024] (grid: 7)
      (8): PairLinear: [1024 -> 1024] (grid: 7)
      (9): PairLinear: [1024 -> 1024] (grid: 7)
    )
    (reducer): Identity()
  )
  (fc): Linear(in_features=1024, out_features=10, bias=True)
)

## Test possible dimensions